# 🎓 RAG Simple - Buscador de Prácticas Loyola

Versión simplificada del sistema RAG **sin chunking**.
- Carga documentos completos
- ChromaDB con limpieza automática
- Prompt simple

In [1]:
import os
import shutil
from pathlib import Path
from dotenv import load_dotenv

from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel

load_dotenv()
print("✅ Dependencias cargadas")

c:\Users\FX506HC\Desktop\PROYECTO_RAG\loyola-finder\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Dependencias cargadas


In [2]:
# Configuración
DATA_DIR = "../data"
CHROMA_DIR = "../chroma_db"

# IMPORTANTE: Limpiar base de datos anterior para evitar duplicados
if os.path.exists(CHROMA_DIR):
    shutil.rmtree(CHROMA_DIR)
    print(f"🗑️  Base de datos anterior eliminada")

print(f"📂 Directorio de datos: {DATA_DIR}")
print(f"💾 Directorio ChromaDB: {CHROMA_DIR}")

📂 Directorio de datos: ../data
💾 Directorio ChromaDB: ../chroma_db


In [3]:
# Cargar documentos COMPLETOS (sin chunking)
loader = DirectoryLoader(
    DATA_DIR, glob="**/*.md", loader_cls=TextLoader, loader_kwargs={"encoding": "utf-8"}
)
documents = loader.load()

# Agregar metadata
for doc in documents:
    file_path = Path(doc.metadata.get("source", ""))
    doc.metadata["company_name"] = file_path.stem

print(f"✅ Documentos cargados: {len(documents)}")
for doc in documents:
    print(f"   - {doc.metadata['company_name']}: {len(doc.page_content)} caracteres")

✅ Documentos cargados: 7
   - Alvaro_Moreno: 1119 caracteres
   - Bida_Farma: 1735 caracteres
   - Endesa: 1586 caracteres
   - Grupo_Hermanos_Martin: 1725 caracteres
   - Heineken: 1741 caracteres
   - Migasa_Aceites: 1830 caracteres
   - Persan: 1587 caracteres


In [4]:
# Modelo de Embeddings
print("🔄 Cargando modelo de embeddings...")
embedding_model = HuggingFaceEmbeddings(
    model_name="paraphrase-multilingual-MiniLM-L12-v2"
)
print("✅ Modelo cargado")

🔄 Cargando modelo de embeddings...


c:\Users\FX506HC\Desktop\PROYECTO_RAG\loyola-finder\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\FX506HC\.cache\huggingface\hub\models--sentence-transformers--paraphrase-multilingual-MiniLM-L12-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


✅ Modelo cargado


In [5]:
# Crear base de datos vectorial CON DOCUMENTOS COMPLETOS
print("🔄 Creando base de datos vectorial...")
vectorstore = Chroma.from_documents(
    documents=documents,  # Sin chunking - documentos completos
    embedding=embedding_model,
    persist_directory=CHROMA_DIR,
)
print(f"✅ Base de datos creada con {len(documents)} documentos")

🔄 Creando base de datos vectorial...
✅ Base de datos creada con 7 documentos


In [6]:
# Configurar LLM
llm = ChatGroq(
    groq_api_key=os.getenv("GROQ_API_KEY"), model_name="llama-3.3-70b-versatile"
)
print("✅ LLM configurado")

✅ LLM configurado


In [7]:
# Retriever simple
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})  # Solo 1 documento
print("✅ Retriever configurado")

✅ Retriever configurado


In [8]:
# Prompt simple
system_prompt = """Eres un asesor de carreras de la Universidad Loyola.
Usa SOLO la información del contexto para responder.
Si el contexto tiene datos sobre una empresa, recomiéndala.

CONTEXTO:
{context}"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("human", "{input}")]
)
print("✅ Prompt configurado")

✅ Prompt configurado


In [9]:
# Cadena RAG
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    RunnableParallel(
        {
            "context": (lambda x: x["input"]) | retriever | format_docs,
            "input": lambda x: x["input"],
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)
print("✅ Cadena RAG construida")

✅ Cadena RAG construida


In [10]:
# TEST: Pregunta sobre moda
query = "¿Qué empresa me recomiendas si me gusta la cerveza?"

print("=" * 60)
print("❓ PREGUNTA:", query)
print("=" * 60)

response = rag_chain.invoke({"input": query})

print("\n💡 RESPUESTA:")
print(response)

❓ PREGUNTA: ¿Qué empresa me recomiendas si me gusta la cerveza?

💡 RESPUESTA:
Si te gusta la cerveza, te recomiendo la empresa **Heineken España SA**. Es una empresa líder en la fabricación y comercialización de cervezas, con una larga trayectoria y una gran variedad de productos. Además, según la información proporcionada, la empresa tiene un gran tamaño y una sólida situación financiera, lo que podría ser atractivo para aquellos que buscan una carrera estable y con oportunidades de crecimiento.


In [11]:
# TEST: Pregunta sobre crecimiento
query2 = "Busco una empresa con buenos resultados financieros"

print("=" * 60)
print("❓ PREGUNTA:", query2)
print("=" * 60)

response2 = rag_chain.invoke({"input": query2})

print("\n💡 RESPUESTA:")
print(response2)

❓ PREGUNTA: Busco una empresa con buenos resultados financieros

💡 RESPUESTA:
Basándome en la información proporcionada, te recomiendo considerar a **Heineken España SA**. Esta empresa presenta algunos indicadores financieros destacados, como:

* Ventas de 1.035.322.984 euros en 2024, lo que sugiere una gran escala de operaciones.
* Un margen EBITDA del 17,9% en 2024, lo que indica una buena gestión de los costos y una alta rentabilidad.
* Un margen neto del 20,1% en 2024, lo que sugiere una alta eficiencia en la generación de beneficios.
* Un total de activos de 1.170.475.863 euros en 2024, lo que indica una gran solidez financiera.
* Un número de empleados de 1.392, lo que sugiere una empresa con una gran capacidad de producción y distribución.

En general, los indicadores financieros de Heineken España SA sugieren que se trata de una empresa con una sólida posición en el mercado y una buena gestión financiera. Sin embargo, es importante tener en cuenta que la información proporciona

In [12]:
# Función simple para consultas
def consultar(pregunta):
    return rag_chain.invoke({"input": pregunta})


# Ejemplo de uso:
consultar("Mi madre es farmaceútica, ¿qué empresa me recomiendas?")

'Considerando que tu madre es farmacéutica, te recomiendo la empresa **Bida Farma**. Esta empresa se dedica a la distribución de productos farmacéuticos y tiene una gran cantidad de empleados (1687) y un capital social significativo (131.183.618 EUR). Esto sugiere que es una empresa estable y con recursos para ofrecer oportunidades laborales y de crecimiento en el sector farmacéutico. Además, su ubicación en Sevilla, España, podría ser conveniente si estás buscando oportunidades en esa región.'